## PROJET HADOOP - MS-SIO-2019 - SNCF - API TRANSILIEN - PARTIE II

#### SPARK STRUCTURED STREAMING (KAFKA CONSUMER)

P. Hamy,  N. Leclercq, L. Poncet - MS-SIO-2019

In [1]:
import os
import json
import time
import logging
from pyspark.sql import SparkSession
import pyspark.sql.types as st
import pyspark.sql.functions as sf

In [2]:
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.ERROR, datefmt='%H:%M:%S')

Changement du logging level afin d'éliminer le bruit généré dans la console par un [_warning_](https://stackoverflow.com/questions/39351690/got-interruptedexception-while-executing-word-count-mapreduce-job) récurrent

In [3]:
log4j = sc._jvm.org.apache.log4j
log4j.LogManager.getRootLogger().setLevel(log4j.Level.ERROR)

#### Création de la session Spark associé au flux Kafka

In [4]:
kafka_session = SparkSession.builder.appName("MS-SIO-HADOOP-PROJECT-STREAM").getOrCreate()

Limitation du nombre de taches lancées par spark (conseil de configutation glané sur internet pour les configurations matérielles les plus modestes).

In [5]:
kafka_session.conf.set('spark.sql.shuffle.partitions', 4)

#### Création du flux Kafka
On utilise ici un [structured spark stream](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html) associé à une source Kafka. 

Il s'agit de spécifier la source via l'adresse du serveur Kafka et le nom du topic auquel on souhaite s'abonner. 

In [6]:
kafka_stream = kafka_session \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "sandbox-hdp.hortonworks.com:6667") \
    .option("subscribe", "transilien-02") \
    .option("startingOffsets", "earliest") \
    .option("kafkaConsumer.pollTimeoutMs", 512) \
    .load()

https://stackoverflow.com/questions/33732346/spark-dataframe-transform-multiple-rows-to-column
https://databricks.com/blog/2018/11/01/sql-pivot-converting-rows-to-columns.html

In [7]:
import random

In [8]:
df = kafka_session.createDataFrame([
    ("s1", "t1", int(time.time() - random.randint(300,1800)), "R"),
    ("s1", "t2", int(time.time() - random.randint(300,1800)), "R"),
    ("s1", "t3", int(time.time() - random.randint(300,1800)), "E"),
    ("s2", "t0", int(time.time() - random.randint(300,1800)), "R"),
    ("s2", "t2", int(time.time() - random.randint(300,1800)), "R"),
    ("s2", "t1", int(time.time() - random.randint(300,1800)), "R"), 
    ("s2", "t3", int(time.time() - random.randint(300,1800)), "R")], 
    ("station", "train", "departure-time", "mode"))

In [9]:
df.show()

+-------+-----+--------------+----+
|station|train|departure-time|mode|
+-------+-----+--------------+----+
|     s1|   t1|    1551609433|   R|
|     s1|   t2|    1551608754|   R|
|     s1|   t3|    1551609436|   E|
|     s2|   t0|    1551609264|   R|
|     s2|   t2|    1551610105|   R|
|     s2|   t1|    1551609589|   R|
|     s2|   t3|    1551608782|   R|
+-------+-----+--------------+----+



In [10]:
reshaped_df = df.groupby('train').pivot('station').max('departure-time').fillna(0).orderBy('train')

In [11]:
reshaped_df.show()

+-----+----------+----------+
|train|        s1|        s2|
+-----+----------+----------+
|   t0|         0|1551609264|
|   t1|1551609433|1551609589|
|   t2|1551608754|1551610105|
|   t3|1551609436|1551608782|
+-----+----------+----------+



#### Schéma de désérialisation des messages  
Les messages injectés dans le flux Kafka sont sérialisés et encodés en binaire dans le champ _value_ du dataframe (format générique des dataframe issus d'un stream Kafka).
```
kafka_stream.printSchema()
root
root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)
 ```
Il est donc nécessaire de spécifier le schéma de désérialisation qui sera passé à la fonction **from_json**.

In [12]:
json_schema = st.StructType(
    [
        st.StructField("station", st.IntegerType(), True),
        st.StructField("train", st.StringType(), True),
        st.StructField("timestamp", st.TimestampType(), True),
        st.StructField("mode", st.StringType(), True),
        st.StructField("mission", st.StringType(), True),
        st.StructField("terminus", st.IntegerType(), True)
    ]
)

A travers, la variable **json_options**, on précise également le format du champ _timestamp_ afin que les valeurs temporelles soient correctement interprétées.

In [13]:
json_options = {"timestampFormat": "yyyy-MM-dd'T'HH:mm:ss.sss'Z'"}

Désérialisation/reformatage des messages.

In [14]:
df = kafka_stream \
    .select(sf.from_json(sf.col("value").cast("string"), json_schema, json_options).alias("departure")) \
    .select("departure.*")

A l'issue de opération le dataframe a le schéma suivant:
```
df.printSchema()
root
 |-- station: integer (nullable = true)
 |-- train: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- mode: string (nullable = true)
 |-- mission: string (nullable = true)
 |-- terminus: integer (nullable = true)
```

Un train apparaitra dans les réponses aux requêtes de l'API SNCF tant que son heure de départ n'appartient pas au passé. On supprime donc les doublons associés aux couples (train, heure de départ). Inutile d'ajouter la station à la contrainte d'exclusion car l'idenfiant d'un train est unique.

In [15]:
df = df.dropDuplicates(["train", "timestamp"])

Les stations contigües A et B (cf. énoncé partie II) 

In [16]:
contiguous_stations = {
    'sa':87381129, # Station A: CLICHY LEVALLOIS
    'sb':87381137  # Station B: ASNIERES SUR SEINE
}

On filtre sur les 'contiguous_stations' et sur le 'mode' de l'horaire de départ de chaque train. On ne retient que les trains au départ des stations qui apparaissent dans la liste _contiguous_stations_ pour lesquels le mode de l'horaire annoncé vaut "R" (horaire réel).

In [17]:
df = df.filter(sf.col("station").isin(list(contiguous_stations.values()))).filter("mode='R'")

conversion de l'heure de départ au format unix timestamp (plus simple à manipuler) - on supprime la colonne "timestamp", devenue inutile.

In [18]:
df = df.withColumn("departure", sf.unix_timestamp("timestamp")).drop("timestamp")

selection des trains dont l'heure de départ est se situe dans l'interval : maintenant +/- (time_window/2) exprimé dqns l'unité unix timestamp (i.e. la seconde)

In [19]:
time_window = 1800

In [20]:
df = df.where(sf.col("departure").between(sf.unix_timestamp(sf.current_timestamp()) - int(time_window/2.), 
                                          sf.unix_timestamp(sf.current_timestamp()) + int(time_window/2.)))

Pseudo aggregation pour obtenir notre ensemble de trains en un _batch_ unique - l'idée est de pouvoir effectuer une requête en mode _complete_ sur notre stream. Il s'agit d'une astuce qui vise à satisfaire une contrainte imposée par Spark : le mode 'complete' ne s'appliquer qu'à des données aggrégées - i.e. issue d'une fonction d'aggrégration de Spark.

In [21]:
df = df.groupBy("train", "station", "departure", "mode", "mission", "terminus").agg(sf.count("train").alias("tmp")).drop("tmp")

In [22]:
query_1 = df \
    .writeStream \
    .format("console") \
    .option("truncate", False) \
    .outputMode("complete") \
    .start()

In [23]:
query_1.stop()

In [24]:
def forEachBatchCallback(batch, batch_number):
    if batch.rdd.isEmpty():
        print(f"ignoring empty batch #{batch_number}")
        return
    t = time.time()
    tmp = batch.groupby("train").pivot("station").max("departure").fillna(0).orderBy("train")
    tmp = tmp.select("train", sf.col(str(contiguous_stations['sa'])).alias("sa"), sf.col(str(contiguous_stations['sb'])).alias("sb"))
    tmp = tmp.filter((sf.col("sa") > 0) & (sf.col("sb") > 0))
    #tmp.show()
    tmp = tmp.withColumn("sa-dpt", sf.from_unixtime(sf.col("sa")))
    tmp = tmp.withColumn("sb-dpt", sf.from_unixtime(sf.col("sb")))
    tmp = tmp.withColumn("dt", sf.col("sb") - sf.col("sa"))
    tmp = tmp.withColumn("now", sf.unix_timestamp(sf.current_timestamp()))
    tmp = tmp.withColumn("in-past", (tmp.now > tmp.sa) & (tmp.now > tmp.sb))
    tmp = tmp.withColumn("in-future", (tmp.now < tmp.sa) & (tmp.now < tmp.sb))
    tmp = tmp.withColumn("in-progress", (sf.col("in-past") != sf.lit(True)) & (sf.col("in-future") != sf.lit(True)))
    #tmp = tmp.filter(sf.col("in-progress") | sf.col("in-future"))
    tmp = tmp.withColumn("departure-uxts", sf.when(sf.col("dt") > sf.lit(0), sf.col("sa")).otherwise(sf.col("sb")))
    tmp = tmp.withColumn("departure-date", sf.when(sf.col("dt") > sf.lit(0), sf.from_unixtime(sf.col("sa"))).otherwise(sf.from_unixtime(sf.col("sb"))))
    #tmp = tmp.withColumn("now - departure", sf.col("now") - sf.col("departure"))
    tmp = tmp.orderBy(sf.col("departure-uxts"))
    #tmp.show()
    tmp = tmp.withColumn("progress", sf.format_number((100. * (sf.col("now") - sf.col("departure-uxts"))) / sf.abs(sf.col("dt")), 1).cast("double"))
    tmp = tmp.withColumn("progress", sf.when(sf.col("progress") < sf.lit(0.), sf.lit(0.)).otherwise(sf.col("progress")))
    tmp = tmp.withColumn("progress", sf.when(sf.col("progress") > sf.lit(100.), sf.lit(100.)).otherwise(sf.col("progress")))
    tmp = tmp.withColumn("direction", sf.when(sf.col("dt") > sf.lit(0.), sf.lit(1)).otherwise(sf.lit(-1)))
    tmp = tmp.withColumn("progress_bar_value", sf.when(sf.col("in-progress"), sf.when(sf.col("dt") > sf.lit(0.), sf.col("progress")).otherwise(100. - sf.col("progress"))).otherwise(sf.col("progress")))
    tmp = tmp.withColumn("progress_bar_value", sf.format_number(sf.col("progress_bar_value"), 1).cast("double"))
    tmp = tmp.join(batch, "train", how="left")
    tmp = tmp.dropDuplicates(["train"])
    tmp = tmp.orderBy(sf.col("departure-uxts"))
    tmp = tmp.select("train", sf.col("departure-date").alias("departure"), "station", "mission", "progress", "direction", "progress_bar_value", "in-past", "in-future", "in-progress")
    tmp.show()
    kafka_session.createDataFrame(tmp.rdd).createOrReplaceTempView("train_progression")
    print(f"`-> took {round(time.time() - t, 2)} s")

In [25]:
query_2 = df \
    .writeStream \
    .foreachBatch(forEachBatchCallback) \
    .outputMode("complete") \
    .start()

+------+-------------------+--------+-------+--------+---------+------------------+-------+---------+-----------+
| train|          departure| station|mission|progress|direction|progress_bar_value|in-past|in-future|in-progress|
+------+-------------------+--------+-------+--------+---------+------------------+-------+---------+-----------+
|133521|2019-03-03 11:40:00|87381137|   VOLA|   100.0|        1|             100.0|   true|    false|      false|
|133548|2019-03-03 11:49:00|87381137|   POVA|   100.0|       -1|             100.0|   true|    false|      false|
|133525|2019-03-03 11:55:00|87381129|   VOLA|     0.8|        1|               0.8|  false|    false|       true|
|133554|2019-03-03 12:03:00|87381137|   POVA|     0.0|       -1|               0.0|  false|     true|      false|
+------+-------------------+--------+-------+--------+---------+------------------+-------+---------+-----------+

`-> took 11.74 s
+------+-------------------+--------+-------+--------+---------+------

Arrêt de la query

In [ ]:
query_1.stop()

In [27]:
query_2.stop()

In [ ]:
kafka_session.streams.active